# May 10, 2025: testing brainrender

In [1]:
import os
import numpy as np
import nibabel as nib
from brainrender import Scene, actor

import pyvista as pv
from vedo import Mesh, settings
settings.default_backend = 'k3d'
from vedo import Volume, show

from skimage.measure import marching_cubes
from tqdm import tqdm
import pandas as pd

In [2]:
class ARGS():
    pass

args = ARGS()
args.SEED = 100

np.random.seed(args.SEED)

In [3]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.makedirs(PARCELS_path, exist_ok=True)

In [4]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 172
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [5]:
TEMP_DESC = (
    f'source-allen' #f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
TEMP_DESC

'source-allen_space-ccfv2_res-200'

In [6]:
SPACING = np.array([args.resolution] * 3)
ORIGIN = np.array([0] * 3)

In [7]:
template_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'
template_img = nib.load(template_file)

In [8]:
parcels_file = f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
parcels_img = nib.load(parcels_file)

try:
    roi_table = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')
    roi_labels = roi_table['roi'].to_numpy()
except:
    roi_labels = np.arange(1, args.num_rois+1)

In [9]:
def random_color(seed=None):
    if seed is not None:
        np.random.seed(seed)
    rgb = np.random.rand(3)  # values between 0 and 1
    rgb_255 = (rgb * 255).astype(int)
    return '#{:02x}{:02x}{:02x}'.format(*rgb_255)

In [10]:
def taubin_smooth_vedo_mesh(mesh: Mesh, n_iter=50, pass_band=0.5) -> Mesh:
    vtk_poly = mesh.dataset
    pv_mesh = pv.wrap(vtk_poly)
    smoothed = pv_mesh.smooth_taubin(n_iter=n_iter, pass_band=pass_band)
    return Mesh(smoothed)

def create_mesh_from_volume(
        volume: np.ndarray,
        threshold: float = 75,
        spacing: tuple = (200, 200, 200),
        origin: tuple = (0, 0, 0),
        smoothing: dict = {'n_iter': 50, 'pass_band': 0.5},
        color: str = 'cornflowerblue',
        alpha: float = 1.0,
) -> Mesh:
    threshold = np.percentile(volume, threshold)
    verts, faces, *_ = marching_cubes(volume.astype(np.uint8), level=threshold)

    verts = np.asarray(spacing) * verts + np.asarray(origin)
    faces = faces.astype(np.uint32)
    faces_vtk = np.c_[np.full(len(faces), 3), faces].flatten()

    mesh = Mesh([verts, faces_vtk])
    mesh = taubin_smooth_vedo_mesh(mesh, **smoothing)
    mesh.color(color).alpha(alpha)
    return mesh

def create_brain_surface(
        template_img: nib.Nifti1Image,
        threshold: float = 75,
        resolution: int = 200,
        smoothing: dict = {'n_iter': 100, 'pass_band': 0.5},
        color: str = '#cccccc',
        alpha: float = 0.1,
) -> Mesh:
    volume = template_img.get_fdata()
    volume = np.transpose(volume, (2, 1, 0))[::-1, :, :]

    spacing = np.array([resolution]*3)
    origin = np.array([0]*3)
    template_mesh = create_mesh_from_volume(
        volume, threshold,
        spacing, origin, 
        smoothing, 
        color, alpha
    )
    
    return template_mesh

def render_roi_vector_on_brain(
        scene: Scene,
        parcels_img: nib.Nifti1Image,
        roi_vector: np.ndarray,
        roi_labels: list,
        threshold: float = 75,
        color: str = 'cornflowerblue',
        use_random_color: bool = False,
        alpha_threshold: float = 0.1,
        resolution: int = 200,
        smoothing: dict = {'n_iter': 50, 'pass_band': 0.5},
):
    assert len(roi_vector) == len(roi_labels), "Mismatch between marginals and labels"

    data = np.round(parcels_img.get_fdata()).astype(int)
    data = np.transpose(data, (2, 1, 0))[::-1, :, :]
    spacing = np.array([resolution] * 3)
    origin = np.array([0]*3)

    for roi_label, membership in tqdm(zip(roi_labels, roi_vector)):
        if membership < alpha_threshold:
            continue

        mask = (data == roi_label).astype(np.uint8)
        if not np.any(mask):
            continue

        # vol = Volume(mask.astype(np.uint8), origin=[0, 0, 0], spacing=spacing)
        # mesh = vol.isosurface()
        # mesh = taubin_smooth_vedo_mesh(mesh, **smoothing)
        # mesh.color(color).alpha(float(membership))
        color = random_color() if use_random_color else color
        mesh = create_mesh_from_volume(
            mask, 
            threshold, 
            spacing, origin, 
            smoothing, 
            color, 
            float(membership),
        )

        scene.add(actor.Actor(mesh, br_class='Volume'))

    return scene


In [11]:
# [random_color() for _ in range(100)]

In [12]:
# pi = np.random.random((args.num_rois, 3))
pi = np.zeros((args.num_rois, 3))
# pi[[80, 303], 1] = 1
# rois = [np.random.randint(0, args.num_rois) for _ in range(170)]
rois = [i for i in range(1, args.num_rois // 1)]
pi[rois, 1] = 1

In [13]:
scene = Scene(atlas_name='allen_mouse_100um')

In [14]:
_ = scene.root.alpha(0.0)
scene.root.alpha()

0.0

In [15]:
template_mesh = create_brain_surface(
    template_img=template_img,
    threshold=75,
    resolution=args.resolution,
)
_ = scene.add(actor.Actor(template_mesh, br_class='Volume'))

In [16]:
scene = render_roi_vector_on_brain(
    scene,
    parcels_img,
    pi[:, 1],
    roi_labels,
    75,
    'cornflowerblue',
    True, # use_random_color = True
    0.9,
    args.resolution,
)

172it [00:00, 210.90it/s]


In [17]:
scene.export(f'test.html')

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

/home/govindas/miniconda3/envs/gt/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


The brainrender scene has been exported for web. The results are saved at test.html

'test.html'